In [7]:
import cv2
import os
import hashlib


In [8]:
# Specify the path to the input image
original_image_path = r"E:\image steganography\code\cleanImage.jpg"

# Specify the path to the encrypted image
encrypted_image_path = r"E:\image steganography\code\encryptedImage.jpg"

In [9]:
def load_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found at {image_path}")
    return img

def hash_password(password):
    return hashlib.sha256(password.encode()).digest()

def initialize_ascii_dictionaries():
    return {chr(i): i for i in range(256)}, {i: chr(i) for i in range(256)}

def encode_message_into_image(img, msg, hashed_password):
    height, width, channels = img.shape
    n = m = z = 0
    
    for i, char in enumerate(msg):
        img[n, m, z] = (int(img[n, m, z]) + d[char] + hashed_password[i % len(hashed_password)]) % 256
        
        m += 1
        if m >= width:
            m = 0
            n += 1
        
        if n >= height:
            raise ValueError("Image too small to hold the entire message.")
        
        z = (z + 1) % 3
    
    return img

def save_image(img,new_filename="encryptedImage.jpg"):
    new_image_path = os.path.join(os.path.dirname(original_image_path), new_filename)
    print(new_image_path)
    cv2.imwrite(new_image_path, img)
    return new_image_path

def decode_message_from_image(img, hashed_password, msg_length):
    height, width, channels = img.shape
    n = m = z = 0
    decoded_chars = []
    
    for i in range(msg_length):
        original_value = (int(img[n, m, z]) - hashed_password[i % len(hashed_password)]+256) % 256
      
        decoded_chars.append(c[original_value])

        
        m += 1
        if m >= width:
            m = 0
            n += 1
        
        if n >= height:
            raise ValueError("Image is too small or corrupted.")
        
        z = (z + 1) % 3
    
    return ''.join(decoded_chars)


In [10]:
# Prompt the user to input the secret message and password
msg = input("Enter secret message: ")
password = input("Enter a passcode: ")

# Hash the password
hashed_password = hash_password(password)
print("Password hashed successfully.",hashed_password)
d, c = initialize_ascii_dictionaries()
print(d,c)

Password hashed successfully. b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'
{'\x00': 0, '\x01': 1, '\x02': 2, '\x03': 3, '\x04': 4, '\x05': 5, '\x06': 6, '\x07': 7, '\x08': 8, '\t': 9, '\n': 10, '\x0b': 11, '\x0c': 12, '\r': 13, '\x0e': 14, '\x0f': 15, '\x10': 16, '\x11': 17, '\x12': 18, '\x13': 19, '\x14': 20, '\x15': 21, '\x16': 22, '\x17': 23, '\x18': 24, '\x19': 25, '\x1a': 26, '\x1b': 27, '\x1c': 28, '\x1d': 29, '\x1e': 30, '\x1f': 31, ' ': 32, '!': 33, '"': 34, '#': 35, '$': 36, '%': 37, '&': 38, "'": 39, '(': 40, ')': 41, '*': 42, '+': 43, ',': 44, '-': 45, '.': 46, '/': 47, '0': 48, '1': 49, '2': 50, '3': 51, '4': 52, '5': 53, '6': 54, '7': 55, '8': 56, '9': 57, ':': 58, ';': 59, '<': 60, '=': 61, '>': 62, '?': 63, '@': 64, 'A': 65, 'B': 66, 'C': 67, 'D': 68, 'E': 69, 'F': 70, 'G': 71, 'H': 72, 'I': 73, 'J': 74, 'K': 75, 'L': 76, 'M': 77, 'N': 78, 'O': 79, 'P': 80, 'Q': 81, 'R': 82, 'S': 83, 'T': 84, 'U': 85, 'V': 86, 'W': 8

In [11]:
# Load the input image
try:
    img = load_image(original_image_path)
    print("Image loaded successfully.")
except FileNotFoundError as e:
    print(e)

# Encode the secret message into the image
try:
    img = encode_message_into_image(img, msg, hashed_password)
    print("Message encoded successfully.")
except ValueError as e:
    print(e)

# Save the modified image to a new file
try:
    encrypted_image_path = save_image(img)
    print(f"Message has been encoded into '{encrypted_image_path}'.")
except Exception as e:
    print(e)

# Open the newly saved encrypted image
#os.startfile(encrypted_image_path)

Image loaded successfully.
Message encoded successfully.
E:\image steganography\code\encryptedImage.jpg
Message has been encoded into 'E:\image steganography\code\encryptedImage.jpg'.


In [12]:
# Load the encrypted image
try:
    img = load_image(encrypted_image_path)
    print("Encrypted image loaded successfully.")
except FileNotFoundError as e:
    print(e)

# Decode the secret message from the image
message_length = len(msg)  # Using the length of the original message
print(message_length)
try:
    decoded_message = decode_message_from_image(img, hashed_password, message_length)
    print("Message decoded successfully.")
    print(f"The secret message is: {decoded_message}")
except ValueError as e:
    print(e)

Encrypted image loaded successfully.
5
Message decoded successfully.
The secret message is: hello
